# **Used Car Price Prediction**

# Business Understanding

## Dataset Description
Source: [Kaggle | Car Price Prediction Challenge](https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge)

This dataset provides detailed information about used cars, including their specifications and pricing details. The dataset consists of approximately 19237 rows and 18 columns, covering various types of used cars such as sedans, SUVs, hatchbacks, etc.

| Column           | Description                                        |
| ---------------- | -------------------------------------------------- |
| ID               | Unique identifier for each car listing.            |
| Price            | Selling price of the car.                          |
| Levy             | Tax applied to importing/exporting the car.        |
| Manufacturer     | Brand or company that produced the car.            |
| Model            | Specific model of the car.                         |
| Prod. year       | Year the car was manufactured.                     |
| Category         | Type of car.                                       |
| Leather interior | Indicates whether the car has leather interior.    |
| Fuel type        | Type of fuel the car uses.                         |
| Engine volume    | Engine displacement.                               |
| Mileage          | Distance the car has traveled.                     |
| Cylinders        | Number of cylinders in the engine.                 |
| Gear box type    | Transmission type.                                 |
| Drive wheels     | Drivetrain configuration.                          |
| Doors            | Number of doors on the car.                        |
| Wheel            | Steering position.                                 |
| Color            | Exterior color of the car.                         |
| Airbags          | Number of airbags installed in the car for safety. |

## Background

The US used car market is influenced by various factors, including age, fuel efficiency, and engine size, all of which contribute to price fluctuations. Buyers and sellers struggle with pricing accuracy, often relying on subjective assessments rather than data-driven insights. By analyzing structured attributes such as mileage, transmission type, etc., machine learning models can offer valuable predictions to improve pricing transparency and decision-making.

## Problem Statement

Using regression-based machine learning techniques, this project aims to develop a predictive model that estimates accurate resale values of used cars based on historical data. The goal is to achieve an RMSE of less than 200 EUR, ensuring reliable pricing predictions. The model will be trained and evaluated within a six-month timeframe, facilitating timely implementation for practical market use. This solution enhances price transparency for buyers and sellers, contributing to greater efficiency and fairness in the US used car market.

## Problem Breakdown


# Library & Function

## Library

In [64]:
# data manipulation
import pandas as pd
import numpy as np
import re

# data viz
import seaborn as sns
import matplotlib.pyplot as plt

## Function

In [65]:
# overview
def check_overview(df):
    '''
    df_overview adalah fungsi yang digunakan untuk melihat informasi seputar dataset.

    Argumen:
    df = dataset yang digunakan.

    Output:
    Informasi overall dataset, missing value, duplicated value dan jumlah unique value setiap kolom.
    '''
    # df overview
    print(df.info())

    # cek missing value
    print(f"\nmissing values: {round(((df.isna().sum().sum())/len(df))*100, 2)}% \n{df.isna().sum()[df.isna().sum()>0]}")

    # cek duplicated value
    print(f"\nduplicated values: {round(((df.duplicated().sum())/len(df))*100,2)}% \n{df.duplicated().sum()}\n")

    # cek nama kolom & jumlah unique value
    for col in df:
        print(f'{col}-#nunique: {df[col].nunique()}')

# Load Data

In [66]:
# load data from csv
df = pd.read_csv('car_price_prediction.csv')
df.tail(7)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
19230,45760891,470,645,TOYOTA,Prius,2011,Hatchback,Yes,Hybrid,1.8,307325 km,4.0,Automatic,Front,04-May,Left wheel,Silver,12
19231,45772306,5802,1055,MERCEDES-BENZ,E 350,2013,Sedan,Yes,Diesel,3.5,107800 km,6.0,Automatic,Rear,04-May,Left wheel,Grey,12
19232,45798355,8467,-,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000 km,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,45778856,15681,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600 km,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,45804997,26108,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,45793526,5331,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258 km,4.0,Automatic,Front,04-May,Left wheel,Black,4
19236,45813273,470,753,HYUNDAI,Sonata,2012,Sedan,Yes,Hybrid,2.4,186923 km,4.0,Automatic,Front,04-May,Left wheel,White,12


Overview:
- The data has 19237 entires and 18 column:
    - Categorical: Manufacturer, Model, Category, Leather interior, Fuel type, Gear box type, Drive wheels, Wheel, and Color.
    - Numerical: Price, Levy, Prod. year, Engine volume, Mileage, Cylinders, Doors, and Airbags.
- Some columns have incorrect data type:
    - levy, Engine volume, Mileage, and Doors should be float.
- Inconsistent column naming. Will be changed to snake_case.
- No missing values were detected, but there is a "-" value in Levy, which could be a non-standard missing value. Another potential non-standard missing value needs to be checked on the other columns, especially the categorical columns.
- 1.63% entries are duplicate. Will be delete.

# Data Cleaning

## Column Name Handling

Column names have Inconsistent type and will be changed to snake_case.

In [67]:
# change col. name type to snake case
df.columns = df.columns.str.lower().str.strip().str.replace('.', '').str.replace(' ', '_')

# delete space in value
df = df.apply(lambda i: i.map(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x) if i.dtype == 'object' else i)

# check
df.columns

Index(['id', 'price', 'levy', 'manufacturer', 'model', 'prod_year', 'category',
       'leather_interior', 'fuel_type', 'engine_volume', 'mileage',
       'cylinders', 'gear_box_type', 'drive_wheels', 'doors', 'wheel', 'color',
       'airbags'],
      dtype='object')

Now, columns have consistent type and more easy to retrieve them.

In [68]:
# save
df_org = df.copy()

In [69]:
# load
df = df_org.copy()

## Duplicate Handling

In [70]:
# drop duplicate
df = df.drop_duplicates(keep='last').reset_index(drop=True)

# check
print('remaining data:',len(df))
print('duplicated data:',df.duplicated().sum())

remaining data: 18924
duplicated data: 0


The duplicate data has been removed and I keep the last to prevent it from losing too much data. 18924 records remaining after deletion.

In [71]:
# save
df_dpl = df.copy()

In [72]:
# load
df = df_dpl.copy()

## Missing Value Handling

### Check on Non-standard Missing Value

In [73]:
# categorical to check
cat_check = ['manufacturer','category','fuel_type','gear_box_type','drive_wheels','color']

# categorical value check
print('Categorical value check')
for i in cat_check:
    # print column and its value
    print(i,df[i].unique())
    print()

# numerical value check
print('Numerical value check')
for i in df.columns:
    # detect unique value
    unique = set(df[i].unique())
    # detect unique value that matched with 0, '-', or '0'
    matched = unique.intersection({0, '-', '0'})

    # print out matched value
    if matched:
        print(i,':',matched)


Categorical value check
manufacturer ['LEXUS' 'HONDA' 'FORD' 'HYUNDAI' 'TOYOTA' 'MERCEDES-BENZ' 'OPEL'
 'PORSCHE' 'JEEP' 'VOLKSWAGEN' 'AUDI' 'RENAULT' 'NISSAN' 'BMW' 'CHEVROLET'
 'SUBARU' 'DAEWOO' 'KIA' 'MITSUBISHI' 'SSANGYONG' 'MAZDA' 'GMC' 'FIAT'
 'INFINITI' 'ALFA ROMEO' 'SUZUKI' 'ACURA' 'LINCOLN' 'VAZ' 'GAZ' 'CITROEN'
 'LAND ROVER' 'MINI' 'DODGE' 'CHRYSLER' 'JAGUAR' 'ISUZU' 'SKODA'
 'DAIHATSU' 'BUICK' 'TESLA' 'CADILLAC' 'PEUGEOT' 'BENTLEY' 'VOLVO' 'სხვა'
 'HAVAL' 'HUMMER' 'SCION' 'UAZ' 'MERCURY' 'ZAZ' 'ROVER' 'SEAT' 'LANCIA'
 'MOSKVICH' 'MASERATI' 'FERRARI' 'SAAB' 'LAMBORGHINI' 'ROLLS-ROYCE'
 'PONTIAC' 'SATURN' 'ASTON MARTIN' 'GREATWALL']

category ['Jeep' 'Hatchback' 'Sedan' 'Microbus' 'Goods wagon' 'Universal' 'Coupe'
 'Minivan' 'Cabriolet' 'Limousine' 'Pickup']

fuel_type ['Hybrid' 'Petrol' 'Diesel' 'CNG' 'Plug-in Hybrid' 'LPG' 'Hydrogen']

gear_box_type ['Automatic' 'Variator' 'Manual' 'Tiptronic']

drive_wheels ['4x4' 'Front' 'Rear']

color ['Silver' 'Black' 'White' 'Grey' 'Blu

Its detected that:
- manufacturer has 'სხვა'.
- category has 'Universal'.
- levy has '-'.
- engine volume has '0'.
- airbags has 0.

those columns need to be checked later to determine whether the value is a missing value or not.

### 'სხვა' in Manufacturer

In [74]:
# missing value in manufacturer
mv_man = df[df['manufacturer'] == 'სხვა']
mv_man

,id,price,levy,manufacturer,model,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
2296,45779593,25089,-,სხვა,IVECO DAYLY,2007,Microbus,No,Diesel,2.3 Turbo,328000 km,4.0,Manual,Rear,04-May,Left wheel,White,1
4678,39223518,9408,-,სხვა,GONOW,2005,Jeep,Yes,Petrol,2.3,102000 km,4.0,Manual,Rear,04-May,Left wheel,Silver,2


**MCAR**   
There is no automobile manufacturer called სხვა. According to Google Translate, "სხვა" is a Georgian word meaning "other". Based on this, I assume that the entry in my dataset is not referring to a specific brand but is likely a placeholder or an indication of missing data.

Looking at the model:
- [Iveco Dayly](https://www.iveco.com/uk/Daily): Found that Iveco is an Italian car brand. However, instead of the Daily, the closest model I found is the Daily.
- [Gonow](https://www.chinamobil.ru/eng/gac/gonow/): Gonow is a brand created by the Guangzhou Automobile Group (GAC), but there is no model information available.

Since we know "სხვა" mean "other", no solid information about the car, and there are only 2 entries, I prefer to delete the entries to ensure data accuracy.

In [75]:
# mv index
idx = mv_man.index.tolist()

# delete missing value
df = df.drop(index=idx).reset_index(drop=True)

# check
df[df['manufacturer'] == 'სხვა']

,id,price,levy,manufacturer,model,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags


In [76]:
# save
df_mv_man = df.copy()

In [82]:
# load
df = df_mv_man.copy()

### 'Universal' in Category

In [78]:
# missing value in category
mv_cat = df[df['category'] == 'Universal']
mv_cat

,id,price,levy,manufacturer,model,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
38,45797488,45734,1091,HYUNDAI,H1,2016,Universal,Yes,Diesel,2.5,61057 km,4.0,Automatic,Front,04-May,Left wheel,Black,4
67,45802120,30063,934,HYUNDAI,H1,2015,Universal,Yes,Diesel,2.5,148105 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
69,45797480,43952,1249,HYUNDAI,H1,2017,Universal,Yes,Diesel,2.5,111643 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
107,45731735,44752,1091,HYUNDAI,H1,2016,Universal,Yes,Diesel,2.5,86000 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
138,45759786,4300,-,MERCEDES-BENZ,208,1985,Universal,No,Diesel,2.5,1111111 km,4.0,Manual,Rear,04-May,Left wheel,Black,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18766,45774661,6429,-,MERCEDES-BENZ,C 200,2000,Universal,No,CNG,2,240000 km,4.0,Manual,Rear,04-May,Left wheel,Blue,4
18821,45731793,41811,1249,HYUNDAI,H1,2017,Universal,Yes,Diesel,2.5,146644 km,4.0,Automatic,Front,04-May,Left wheel,Black,4
18844,45744959,17249,1054,MERCEDES-BENZ,E 220,2009,Universal,Yes,Diesel,2.2 Turbo,172000 km,4.0,Tiptronic,Rear,04-May,Left wheel,White,10
18891,45809497,22075,503,TOYOTA,Aqua,2012,Universal,Yes,Hybrid,1.5,52456 km,4.0,Automatic,Front,04-May,Left wheel,White,4


**MCAR**   
According to [Car and Driver](https://www.caranddriver.com/shopping-advice/g26100588/car-types/), "universal" is not a car body type. For example, the [Hyundai H1](https://www.hyundai.com/worldwide/en/commercial/h-1/highlights) is a van and the [Toyota Aqua](https://global.toyota/en/mobility/toyota-brand/gallery/aqua.html) is a hatchback. Because there is no pattern related to the "universal" category, I assume "universal" is likely a placeholder for missing data.

To handle universal, the value will be changed with their group mode and if there is a data with only 1 entries, the category will become 'Unknown'.

In [83]:
# filter selain universal
df_no_univ = df[df['category']!='Universal']

# group by category to search for mode
group_cat = df_no_univ.groupby(['manufacturer', 'model'])['category'].agg(lambda x: x.mode()[0])

# convert to dataframe
group_cat = group_cat.reset_index(name='mode')

# merge into dataframe
df = df.merge(group_cat, on=['manufacturer','model'], how='left')

# replace universal in category with NA
df['category'] = df['category'].replace('Universal',np.nan)

# replace missing value with mode column and if still missing replace with 'Unknown'
df['category'] = df['category'].fillna(df['mode']).fillna('Unknown')

# make sure if every missing value in mode followed by unknown category
check_condition = (df['mode'].isna() & (df['category']!='Unknown')).any()

# if not
if check_condition:
    print('FALSE')
# if yes
else:
    print(f'TRUE, {df[df['category'] == 'Unknown'].shape[0]} Unknwon values')

TRUE, 27 Unknwon values


In [86]:
# remove mode column
df = df.drop(columns='mode')

# save
df_mv_cat = df.copy()

In [87]:
# load
df = df_mv_cat.copy()

### '-' in Levy

In [89]:
# missing value in levy
mv_levy = df[df['levy'] == '-']
mv_levy

,id,price,levy,manufacturer,model,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
1,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
8,45756839,26657,-,LEXUS,RX 350,2007,Jeep,Yes,Petrol,3.5,128500 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
10,45814819,8781,-,FORD,Transit,1999,Microbus,No,CNG,4,0 km,8.0,Manual,Rear,02-Mar,Left wheel,Blue,0
11,45815568,3000,-,OPEL,Vectra,1997,Goods wagon,No,CNG,1.6,350000 km,4.0,Manual,Front,04-May,Left wheel,White,4
21,45814106,7840,-,FORD,Transit,2001,Microbus,No,Diesel,2.0 Turbo,230000 km,4.0,Manual,Front,02-Mar,Left wheel,White,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18908,44571028,8781,-,DODGE,Caliber,2008,Hatchback,No,Petrol,2.4,0 km,4.0,Automatic,Front,04-May,Left wheel,Blue,4
18909,45768089,19130,-,KIA,Optima ex,2014,Sedan,Yes,Petrol,2.4,35800 km,4.0,Automatic,Front,04-May,Left wheel,Black,12
18911,45796788,7840,-,NISSAN,Skyline,2003,Sedan,Yes,Petrol,3,220000 km,6.0,Tiptronic,Rear,04-May,Right-hand drive,White,0
18914,39977395,50,-,TOYOTA,Prius,2008,Hatchback,No,Hybrid,1.5,150000 km,4.0,Automatic,Front,04-May,Left wheel,Silver,6


**MAR**   


To do:
- ~~Change column name to snake_case~~
~~Delete duplicate~~
- Check another potential non-standard missing value.
    - ~~manufacturer has 'სხვა'.~~
    - ~~category has 'Universal'.~~
    - levy has '-'.
    - engine volume has '0'.
    - airbags has 0.
- Change levy, Engine volume, Mileage, and Doors to float